In [0]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torchsummary import summary

In [4]:
print("--sys.version--")
print(sys.version)

print("\n--torch.__version__--")
print(torch.__version__)

--sys.version--
3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]

--torch.__version__--
1.1.0


In [0]:
batch_size = 20
total_epoch = 100
learning_rate = 0.01
use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/train", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/test", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
def train(model, train_loader):
  model.train()
  
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
  losses = []
  for i, (image, label) in enumerate(train_loader):
    
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
    pred_label = model(image)
    loss = criterion(pred_label, label)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  avg_loss = sum(losses)/len(losses)
  return avg_loss

In [0]:
def eval(model, test_loader):
  model.eval()
  device = next(model.parameters()).device.index
  
  total_cnt = 0
  correct_cnt = 0
  
  for i, (image, label) in enumerate(test_loader):
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
      out = model(image)
      _, pred_label = torch.max(out.data, 1)
      total_cnt += image.data.size()[0]
      correct_cnt += (pred_label == label.data).sum().item()
      
    return correct_cnt / total_cnt

In [0]:
class SimpleMLP(nn.Module):
  def __init__(self):
    super(SimpleMLP, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.ReLU()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.ReLU()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.ReLU()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.ReLU()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleMLP_Sigmoid(nn.Module):
  def __init__(self):
    super(SimpleMLP_Sigmoid, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.Sigmoid()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.Sigmoid()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.Sigmoid()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.Sigmoid()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    # Convolution layer
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2)
    
    self.conv2 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2)
    
    self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
    self.act3 = nn.ReLU()
    
    self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
    self.act4 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2)
    
    # Fully-Connected layer
    self.fc1 = nn.Linear(256 * 2 * 2, 1000)
    self.act5 = nn.ReLU()
    self.output = nn.Linear(1000, 10)
    
    
  def forward(self, x):
    x = self.pool1(self.act1(self.conv1(x)))
    x = self.pool2(self.act2(self.conv2(x)))
    x = self.act3(self.conv3(x))
    x = self.act4(self.conv4(x))
    x = self.pool3(x)
    
    x = x.view(-1, 256 * 2 * 2)
    
    x = self.act5(self.fc1(x))
    out = self.output(x)
    return out

In [0]:
class SimpleVGG(nn.Module):
  def __init__(self):
    super(SimpleVGG, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), padding=(1,1))
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    
    self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3), padding=(1,1))
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_1 = nn.ReLU()
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_2 = nn.ReLU()
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_1 = nn.ReLU()
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_2 = nn.ReLU()
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_3 = nn.ReLU()
    self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
    
    # Output layer
    self.fc1 = nn.Linear(512 * 2 * 2, 512)
    self.act5 = nn.ReLU()
    self.out = nn.Linear(512, 10)
    
  def forward(self, x):
    x1 = x
    x2 = self.act1(self.conv1(x1))
    x3 = self.pool1(x2)
    
    x4 = self.act2(self.conv2(x3))
    x5 = self.pool2(x4)
    
    x6 = self.act3_1(self.conv3_1(x5))
    x7 = self.act3_2(self.conv3_2(x6))
    x8 = self.act3_3(self.conv3_3(x7))
    x9 = self.pool3(x8)
    
    x10 = self.act4_1(self.conv4_1(x9))
    x11 = self.act4_2(self.conv4_2(x10))
    x12 = self.act4_3(self.conv4_3(x11))
    x13 = self.pool4(x12)
    
    x14 = x13.view(-1, 512 * 2 * 2)
    
    x15 = self.act5(self.fc1(x14))
    
    out = self.out(x15)
    return out

In [0]:
class SimpleResNet(nn.Module):
  def __init__(self):
    super(SimpleResNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=(3,3), padding=(1,1))
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    
    self.conv2 = nn.Conv2d(64, 128, kernel_size=(3,3),padding=(1,1))
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_1 = nn.ReLU()
    self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_2 = nn.ReLU()
    self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3,3), padding=(1,1))
    self.act3_3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
    
    self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_1 = nn.ReLU()
    self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_2 = nn.ReLU()
    self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
    self.act4_3 = nn.ReLU()
    self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
    
    # Output layer
    self.fc1 = nn.Linear(512 * 2 * 2, 512)
    self.act5 = nn.ReLU()
    self.out = nn.Linear(512, 10)
  
  def forward(self, x):
    x1 = x
    x2 = self.act1(self.conv1(x1))
    x3 = self.pool1(x2)
    
    x4 = self.act2(self.conv2(x3))
    x5 = self.pool2(x4)
    
    x6 = self.act3_1(self.conv3_1(x5))
    x7 = self.act3_2(self.conv3_2(x6))
    x8 = self.act3_3(self.conv3_3(x7) + x6)
    x9 = self.pool3(x8)
    
    x10 = self.act4_1(self.conv4_1(x9))
    x11 = self.act4_1(self.conv4_2(x10))
    x12 = self.act4_1(self.conv4_3(x11) + x10)
    x13 = self.pool4(x12)
    
    x14 = x13.view(-1, 512 * 2 * 2)
    
    x15 = self.act5(self.fc1(x14))
    
    out = self.out(x15)
    return out           

In [15]:
mlp_model = SimpleMLP().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(mlp_model, input_size = (3, 32, 32))

1 loss : 2.28015456199646
Accuracy : 0.5
2 loss : 2.229531097412109
Accuracy : 0.5
3 loss : 2.1809930324554445
Accuracy : 0.5
4 loss : 2.128932809829712
Accuracy : 0.5
5 loss : 2.0688305377960203
Accuracy : 0.5
6 loss : 1.9958940267562866
Accuracy : 0.5
7 loss : 1.900928831100464
Accuracy : 0.5
8 loss : 1.7741100549697877
Accuracy : 0.5
9 loss : 1.600469183921814
Accuracy : 0.5
10 loss : 1.3800851345062255
Accuracy : 0.5
11 loss : 1.1585801839828491
Accuracy : 0.5
12 loss : 0.9778548836708069
Accuracy : 0.5
13 loss : 0.8624639749526978
Accuracy : 0.7
14 loss : 0.7890028476715087
Accuracy : 0.5
15 loss : 0.7449694752693177
Accuracy : 0.55
16 loss : 0.7118423581123352
Accuracy : 0.6
17 loss : 0.6924200892448426
Accuracy : 0.7
18 loss : 0.6970789194107055
Accuracy : 0.4
19 loss : 0.6569279313087464
Accuracy : 0.5
20 loss : 0.6405163645744324
Accuracy : 0.8
21 loss : 0.6604423761367798
Accuracy : 0.65
22 loss : 0.6222152709960938
Accuracy : 0.5
23 loss : 0.6170679569244385
Accuracy : 0.6
2

In [16]:
mlp_model2 = SimpleMLP_Sigmoid().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(mlp_model2, input_size = (3, 32, 32))

1 loss : 0.011597018409520387
Accuracy : 0.8
2 loss : 0.011286735534667969
Accuracy : 0.8
3 loss : 0.011541843600571155
Accuracy : 0.8
4 loss : 0.010806656070053577
Accuracy : 0.8
5 loss : 0.01031023021787405
Accuracy : 0.8
6 loss : 0.009770236164331435
Accuracy : 0.8
7 loss : 0.009912171214818955
Accuracy : 0.8
8 loss : 0.009347825031727552
Accuracy : 0.8
9 loss : 0.009188704565167427
Accuracy : 0.8
10 loss : 0.008737277984619141
Accuracy : 0.8
11 loss : 0.008673772867769002
Accuracy : 0.8
12 loss : 0.00842550741508603
Accuracy : 0.8
13 loss : 0.008053269516676665
Accuracy : 0.8
14 loss : 0.007808590028434992
Accuracy : 0.8
15 loss : 0.007631077663972974
Accuracy : 0.8
16 loss : 0.007739477045834065
Accuracy : 0.8
17 loss : 0.007466812245547771
Accuracy : 0.8
18 loss : 0.007018232252448797
Accuracy : 0.8
19 loss : 0.007039752043783665
Accuracy : 0.8
20 loss : 0.006739058624953031
Accuracy : 0.8
21 loss : 0.006733107566833496
Accuracy : 0.8
22 loss : 0.006381220929324627
Accuracy : 0.8

In [17]:
cnn_model = SimpleCNN().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(cnn_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(cnn_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(cnn_model, input_size = (3,32,32))

1 loss : 2.2888842582702638
Accuracy : 0.5
2 loss : 2.2410012245178224
Accuracy : 0.5
3 loss : 2.1907605648040773
Accuracy : 0.5
4 loss : 2.1325640201568605
Accuracy : 0.5
5 loss : 2.0566579818725588
Accuracy : 0.5
6 loss : 1.9420445919036866
Accuracy : 0.55
7 loss : 1.7396649837493896
Accuracy : 0.4
8 loss : 1.360076117515564
Accuracy : 0.5
9 loss : 0.9619580626487731
Accuracy : 0.5
10 loss : 0.7870545029640198
Accuracy : 0.5
11 loss : 0.7320159196853637
Accuracy : 0.5
12 loss : 0.7154817700386047
Accuracy : 0.5
13 loss : 0.713418447971344
Accuracy : 0.5
14 loss : 0.7434101939201355
Accuracy : 0.5
15 loss : 0.6996376156806946
Accuracy : 0.5
16 loss : 0.7255079627037049
Accuracy : 0.5
17 loss : 0.725289237499237
Accuracy : 0.5
18 loss : 0.7065657258033753
Accuracy : 0.5
19 loss : 0.7306170582771301
Accuracy : 0.5
20 loss : 0.7035523295402527
Accuracy : 0.5
21 loss : 0.6955435752868653
Accuracy : 0.5
22 loss : 0.706280529499054
Accuracy : 0.5
23 loss : 0.7011498808860779
Accuracy : 0.5


In [18]:
vgg_model = SimpleVGG().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(vgg_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(vgg_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(vgg_model, input_size = (3,32,32))

1 loss : 2.2813321113586427
Accuracy : 0.5
2 loss : 2.2572453022003174
Accuracy : 0.5
3 loss : 2.233183431625366
Accuracy : 0.5
4 loss : 2.209451103210449
Accuracy : 0.5
5 loss : 2.1861791133880617
Accuracy : 0.5
6 loss : 2.162987470626831
Accuracy : 0.5
7 loss : 2.140240716934204
Accuracy : 0.5
8 loss : 2.117628335952759
Accuracy : 0.5
9 loss : 2.0953235149383547
Accuracy : 0.5
10 loss : 2.073278045654297
Accuracy : 0.5
11 loss : 2.051110315322876
Accuracy : 0.5
12 loss : 2.0294042110443113
Accuracy : 0.5
13 loss : 2.007502222061157
Accuracy : 0.5
14 loss : 1.9857707262039184
Accuracy : 0.5
15 loss : 1.9642173290252685
Accuracy : 0.5
16 loss : 1.9428031682968139
Accuracy : 0.5
17 loss : 1.9213690042495728
Accuracy : 0.5
18 loss : 1.899807906150818
Accuracy : 0.5
19 loss : 1.8783287763595582
Accuracy : 0.5
20 loss : 1.8566905975341796
Accuracy : 0.5
21 loss : 1.8351734399795532
Accuracy : 0.5
22 loss : 1.8136045217514039
Accuracy : 0.5
23 loss : 1.7919514894485473
Accuracy : 0.5
24 los

In [19]:
resnet_model = SimpleResNet().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(resnet_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(resnet_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(resnet_model, input_size = (3,32,32))

1 loss : 2.277858018875122
Accuracy : 0.5
2 loss : 2.2413113117218018
Accuracy : 0.5
3 loss : 2.204602527618408
Accuracy : 0.5
4 loss : 2.166339111328125
Accuracy : 0.5
5 loss : 2.1241406917572023
Accuracy : 0.5
6 loss : 2.073292541503906
Accuracy : 0.5
7 loss : 2.0073747634887695
Accuracy : 0.5
8 loss : 1.9103617668151855
Accuracy : 0.5
9 loss : 1.7436917304992676
Accuracy : 0.5
10 loss : 1.417808437347412
Accuracy : 0.5
11 loss : 0.9863250017166137
Accuracy : 0.5
12 loss : 0.780078125
Accuracy : 0.5
13 loss : 0.7425066828727722
Accuracy : 0.5
14 loss : 0.7628249049186706
Accuracy : 0.5
15 loss : 0.7793685555458069
Accuracy : 0.5
16 loss : 0.7298925161361695
Accuracy : 0.5
17 loss : 0.73058522939682
Accuracy : 0.5
18 loss : 0.7110967040061951
Accuracy : 0.4
19 loss : 0.7101500630378723
Accuracy : 0.5
20 loss : 0.7189873456954956
Accuracy : 0.5
21 loss : 0.7063756227493286
Accuracy : 0.4
22 loss : 0.7353615522384643
Accuracy : 0.5
23 loss : 0.7034193873405457
Accuracy : 0.5
24 loss : 0